In [1]:
import pennylane as qml
import pennylane.numpy as np

In [3]:
#qml.matrix(qml.RY(0.2*np.pi,0)@qml.PauliX(0)@qml.RY(-0.2*np.pi,0))

array([[-0.58778525+0.j,  0.80901699+0.j],
       [ 0.80901699+0.j,  0.58778525+0.j]])

In [6]:
X=(np.load('./saved_data_PneumoniaMNIST/train_seqs_42.npz')['arr_0']) # train images

train_label=np.load('./saved_data_PneumoniaMNIST/train_labels.npz')['arr_0']


Y = np.array([int(y) for y in train_label]) # train lables
Y = Y * 2 - np.ones(len(Y)) # shift label from {0, 1} to {-1, 1}

In [4]:
#qml.matrix(qml.RY(0.2*np.pi,0)@qml.RY(-0.2*np.pi,0))

array([[1.00000000e+00+0.j, 1.19889621e-17+0.j],
       [1.19889621e-17+0.j, 1.00000000e+00+0.j]])

In [3]:
index_x=['x0','x1','x2','x3','x4']
index_y=['y0','y1','y2','y3','y4']
def encoder(image):
    for wire in (index_x+index_y):
        qml.Hadamard(wires=wire)
    for i in range(len(image)):
        y = bin(i)[2:]
        y = '0'*(5-len(y)) + y
        for j in range(len(image[0])):
            if image[i][j]>0.001:
                x = bin(j)[2:]
                x = '0'*(5-len(x)) + x
                qml.RY(image[i][j]/2*np.pi,wires='gray')
                qml.MultiControlledX(control_wires=(index_x+index_y), wires='gray', control_values=x+y)
                qml.RY(image[i][j]/2*np.pi,wires='gray')
            
            

In [7]:
def entangling_layer0(input_wires,output_wires,weights):
    
    weights_ = (weight for weight in weights)
    for i in input_wires:
        for j in output_wires:
            qml.CNOT(wires=[i,j])
            
            param = next(weights_)
            qml.RZ(param, wires=j)
            
            param = next(weights_)
            qml.RX(param, wires=j) 
            
            qml.CNOT(wires=[i,j])
            


In [8]:
def entangling_layer1(wires,weights):
    n=len(wires)
    weights_ = (weight for weight in weights)
    for i in range(1,n):
        j = 0
        r = []
        while j <= n-1:
            if j not in r:
                qml.CNOT([wires[j],wires[j+i]])
                r += [j+1]
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([wires[j],wires[j+i]])
                j += 1
        for j in r:
            qml.CNOT([wires[j],wires[j+i]])
            
            param = next(weights_)
            qml.RZ(param, wires=j)
            
            param = next(weights_)
            qml.RX(param, wires=j) 
            
            qml.CNOT([wires[j],wires[j+i]])

In [14]:
'''
wires = [0,1,2,3]
np.random.seed(0)
weights= 0.01 * np.random.randn(12, requires_grad=True)

dev = qml.device("default.qubit", wires=wires)
@qml.qnode(dev)

def entangling_layer1(wires,weights):
    n=len(wires)
    weights_ = (weight for weight in weights)
    for i in range(1,n):
        j = 0
        r = []
        while j <= n-i-1:
            if j not in r:
                qml.CNOT([wires[j],wires[j+i]])
                r += [j+1]
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([wires[j],wires[j+i]])
            j += 1
        for j in r:
            if j+i <= n-1:
                qml.CNOT([wires[j],wires[j+i]])
            
                param = next(weights_)
                qml.RZ(param, wires=j)
            
                param = next(weights_)
                qml.RX(param, wires=j) 
            
                qml.CNOT([wires[j],wires[j+i]])
    return qml.probs(wires[0])
            
print(qml.draw(entangling_layer1)(wires,weights))'''

0: ─╭●──RZ(0.02)──RX(0.00)─╭●────────────────────────────╭●──RZ(0.01)──RX(-0.00)─╭●──────────────
1: ─╰X─────────────────────╰X─╭●──RZ(0.02)──RX(-0.01)─╭●─│───────────────────────│──╭●──RZ(-0.00)
2: ─╭●──RZ(0.01)──RX(0.02)─╭●─╰X──────────────────────╰X─╰X──────────────────────╰X─│────────────
3: ─╰X─────────────────────╰X───────────────────────────────────────────────────────╰X───────────

───────────────╭●──RZ(0.00)──RX(0.01)─╭●─┤  Probs
───RX(0.00)─╭●─│──────────────────────│──┤       
────────────│──│──────────────────────│──┤       
────────────╰X─╰X─────────────────────╰X─┤       


In [9]:
# The training circuit
encoder_wires = ['x0','x1','x2','x3','x4','y0','y1','y2','y3','y4','gray','train']
dev = qml.device("default.qubit", wires=encoder_wires)
@qml.qnode(dev)
def circuit(weights, image):
    encoder(image)
    entangling_layer0(['x0','x1','x2','x3','x4','y0','y1','y2','y3','y4','gray'],['train'],weights)
    #entangling_layer1(['x0','x1','x2','x3','x4','y0','y1','y2','y3','y4','gray'],weights)
    return qml.expval(qml.PauliZ('train'))



In [10]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [11]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [12]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [13]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [14]:
np.random.seed(0)
weights_init = 0.01 * np.random.randn(22, requires_grad=True)
#weights_init = 0.01 * np.random.randn(110, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

In [15]:
from pennylane.optimize import NesterovMomentumOptimizer
opt = NesterovMomentumOptimizer(0.5)
batch_size = 2

In [16]:
# Training the circuit
weights = weights_init
bias = bias_init
for it in range(2):

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# test model with test dataset

test_X=(np.load('images.npy')*255)[495:505] # train images

test_label=np.load('labels.npy')[495:505]
test_Y = np.array([int(y) for y in test_label]) # train lables
test_Y = test_Y * 2 - np.ones(len(test_Y)) # shift label from {0, 1} to {-1, 1}

In [ ]:
test_predictions = [np.sign(variational_classifier(weights, bias, x)) for x in test_X]
test_acc = accuracy(test_Y, test_predictions)